### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings 
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE, SequentialFeatureSelector

### Funcion: super_selector 

Esta función debe recibir como argumento un dataframe de features "dataset", un argumento "target_col" (que puede hacer referencia a una feature numérica o categórica) que puede ser "", un argumento "selectores" de tipo diccionario que puede estar vacío, y un argumento "hard_voting" como una lista vacía. 

CAUSISTICA y funcionamiento:

* Si target_col no está vacío y es un columna válidad del dataframe, la función comprobará el valor de "selectores":
    * Si "selectores" es un diccionario vacío o None:
        La fución devuelve una lista con todas las columnas del dataframe que no sean el target, tengan un valor de cardinalidad diferente del 99.99% (no sean índices) y no tengan un único valor.
    * Si "selectores" no es un diccionario vacío, espera encontrar las siguientes posibles claves (y actúa en consecuencia):  
        "KBest": Tendrá como valor el número de features a seleccionar aplicando un KBest. La función debe crear una lista con las features obtenidas de emplear un SelectKBest con ANOVA.  
        "FromModel": Tendrás como valores una lista con dos elementos, el primero la instancia de un modelo de referencia y el segundo un valor entero o compatible con el argumento "threshold" de SelectFromModel de sklearn. En este caso la función debe crear un a lista con las features obtenidas de aplicar un SelectFromModel con el modelo de referencia, y utilizando "threshold" con el valor del segundo elemento si este no es un entero. En este caso, cuando sea un entero, usarás SelectFromModel con los argumentos "max_features" igual al valor del segundo elemento y "threshold" igual a -np.inf. (Esto hace que se seleccionen "max_features" features)  
        "RFE": Tendrá como valor una tupla con tres elementos. El primero será un modelo instanciado, el segundo elemento determina el número de features a seleccionar y el tercero el step a aplicar. Serán los tres argumentos del RFE de sklearn que usará la función para generar una lista de features.  
        "SFS": Tendrá como valor un tupla con 2 elementos, el modelo de referencia instanciado y el numero de featureas a alcanzar. Esta vez la función empleará un SFS para obtener las lista de features seleccionadas.

* La función debe devolver tantas listas seleccionadas como claves en el diccionario de selectores y una adicional con el resultado de aplicar un hard voting a las listas obtenidas de aplicar el diccionario "selectores" y las que contenga "hard_voting", en caso de que "hard_voting" contenga una o más listas. La función devolverá un diccionario con claves equivalentes a las de selectores pero con la lista correspondiente asignada a cada clave y una adicional "hard_voting" caso de que "hard_voting" como argumento no sea una lista vacía.

Ejemplo:

```python
selectores = {
    "KBest": 5,
    "FromModel": [RandomForestClassifier(),5],
    "RFE": [LogisticRegression(),5,1]
}
super_selector(train_set_titanic, target_col = "Survived", selectores = selectores, hard_voting = ["Pclass","who","embarked_S","fare","age"])

```

Devolvera un diccionario del tipo: 
```python
{
    "KBest": [lista de features obtenidas con un SelectKBest(f_classif, k=5) con fit a train_set_titanic y target_col, sin la target_col en train_set_titanic, claro],
    "FromModel": [lista de features obtenidas de aplicar un SelecFromModel con el RandomForestClassfier y max_features = 5 y threshold = -np.inf],
    "RFE": [lista de features obtenidas de un RFE con argumentos el LogisticRegressor, n_features_to_select = 5, y step = 1],
    "hard_voting": [lista con las len(hard_voting) features con más votos entre las cuatro listas]
}
```
NOTA: Si hard_voting esta a [], la función sigue devolviendo el hard_voting pero sólo con las listas creadas internamente (si hay una sola también), es decir que la función siempre devuelve al menos dos listas.

In [2]:
def super_selector(dataset, target_col = "", selectores = None, hard_voting = []):

    """
    Función que selecciona features de un dataframe utilizando varios métodos y realiza un hard voting entre las listas seleccionadas
    
    Argumentos:
    dataset (pd.DataFrame): DataFrame con las features y el target
    target_col (str): Columna objetivo en el dataset. Puede ser numérica o categórica
    selectores (dict): Diccionario con los métodos de selección a utilizar. Puede contener las claves "KBest", "FromModel", "RFE" y "SFS"
    hard_voting (list): Lista de features para incluir en el hard voting

    Retorna:
    dict: Diccionario con las listas de features seleccionadas por cada método y una lista final por hard voting
    """
    
    if selectores is None:
        selectores = {}
    
    features = dataset.drop(columns = [target_col]) if target_col else dataset
    target = dataset[target_col] if target_col else None
    
    result = {}

    # Caso en que selectores es vacío o None
    if target_col and target_col in dataset.columns:
        if not selectores:
            filtered_features = [col for col in features.columns if
                                 (features[col].nunique() / len(features) < 0.9999) and
                                 (features[col].nunique() > 1)]
            result["all_features"] = filtered_features

    # Aplicación de selectores si no es vacío
    if selectores:
        if "KBest" in selectores:
            k = selectores["KBest"]
            selector = SelectKBest(score_func = f_classif, k = k)
            selector.fit(features, target)
            selected_features = features.columns[selector.get_support()].tolist()
            result["KBest"] = selected_features

        if "FromModel" in selectores:
            model, threshold_or_max = selectores["FromModel"]
            if isinstance(threshold_or_max, int):
                selector = SelectFromModel(model, max_features = threshold_or_max, threshold = -np.inf)
            else:
                selector = SelectFromModel(model, threshold = threshold_or_max)
            selector.fit(features, target)
            selected_features = features.columns[selector.get_support()].tolist()
            result["FromModel"] = selected_features

        if "RFE" in selectores:
            model, n_features, step = selectores["RFE"]
            selector = RFE(model, n_features_to_select = n_features, step = step)
            selector.fit(features, target)
            selected_features = features.columns[selector.get_support()].tolist()
            result["RFE"] = selected_features

        if "SFS" in selectores:
            model, k_features = selectores["SFS"]
            sfs = SequentialFeatureSelector(model, n_features_to_select = k_features, direction = "forward")
            sfs.fit(features, target)
            selected_features = features.columns[sfs.get_support()].tolist()
            result["SFS"] = selected_features

    # Hard Voting
    if hard_voting or selectores:
        voting_features = []
        if "hard_voting" not in result:
            voting_features = hard_voting.copy()
        for key in result:
            voting_features.extend(result[key])

        feature_counts = pd.Series(voting_features).value_counts()
        hard_voting_result = feature_counts[feature_counts > 1].index.tolist()
        
        result["hard_voting"] = hard_voting_result if hard_voting_result else list(feature_counts.index)

    return result

### Pruebas

In [3]:
# Cargar dataframe
titanic = pd.read_csv("./data/titanic.csv")

# Transformar variables categóricas
titanic = pd.get_dummies(titanic, drop_first = True)

# Definir la columna objetivo
target_col = 'alive_yes'

# Separar las features y el target
features = titanic.drop(columns = [target_col])
target = titanic[target_col]

In [4]:
selectores = {
    "KBest" : 5,
    "FromModel" : (RandomForestClassifier(), 5),
    "RFE" : (RandomForestClassifier(), 5, 1),
    "SFS" : (RandomForestClassifier(), 5)
}

resultados = super_selector(titanic, target_col = target_col, selectores = selectores)
resultados

{'KBest': ['adult_male', 'sex_male', 'class_Third', 'who_man', 'who_woman'],
 'FromModel': ['age', 'fare', 'adult_male', 'sex_male', 'who_man'],
 'RFE': ['age', 'fare', 'adult_male', 'class_Third', 'who_man'],
 'SFS': ['fare',
  'adult_male',
  'class_Second',
  'class_Third',
  'embark_town_Queenstown'],
 'hard_voting': ['adult_male',
  'class_Third',
  'who_man',
  'fare',
  'sex_male',
  'age']}